In [ ]:
!unzip /content/converted_savedmodel.zip -d /content/

Archive:  /content/converted_savedmodel.zip
   creating: /content/model.savedmodel/
 extracting: /content/model.savedmodel/saved_model.pb  
   creating: /content/model.savedmodel/assets/
   creating: /content/model.savedmodel/variables/
 extracting: /content/model.savedmodel/variables/variables.index  
 extracting: /content/model.savedmodel/variables/variables.data-00000-of-00001  
 extracting: /content/labels.txt     


In [ ]:

# ================================
# 1. Librerías
# ================================
import os
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)

# ================================
# 2. Cargar modelo SavedModel (local)
# ================================
# Carga el SavedModel exportado por el entrenamiento local en 'model_from_src'
local_model_path = os.path.join(os.getcwd(), "model_from_src")
fallback_model_path = "/content/model"  # por compatibilidad si usas Colab

model_path = local_model_path if tf.io.gfile.exists(local_model_path) else fallback_model_path
try:
    model = tf.saved_model.load(model_path)
    print("SavedModel cargado desde:", model_path)
except Exception as e:
    print(f"Error cargando SavedModel: {e}")
    model = None

infer = None
if model is not None:
    try:
        infer = model.signatures["serving_default"]
        print("Usando firma 'serving_default'.")
        print("Firma de entrada esperada:")
        for input_name, input_spec in infer.structured_input_signature[1].items():
            print(f"  Entrada '{input_name}': {input_spec}")
    except Exception as e:
        print("No se pudo obtener la firma:", e)

# ================================
# 3. Leer imagen (de src/ si existe)
# ================================
DATA_DIR = os.path.join(os.getcwd(), "src")
img_path = None

if os.path.isdir(DATA_DIR):
    # Busca una imagen cualquiera dentro de las subcarpetas de src/
    exts = (".jpg", ".jpeg", ".png", ".bmp")
    for root, _, files in os.walk(DATA_DIR):
        for f in files:
            if f.lower().endswith(exts):
                img_path = os.path.join(root, f)
                break
        if img_path:
            break

if img_path is None:
    # Fallback a la ruta usada por el profesor en Colab
    img_path = "/content/manita.png"

print("Imagen a usar:", img_path)
img = cv2.imread(img_path)

if img is None:
    print(f"Error: no se pudo cargar la imagen: {img_path}")
else:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

    # ================================
    # 4. Preparar la imagen para el modelo
    # ================================
    target_size = (224, 224)  # alto x ancho esperado por MobileNetV2
    img_resized = cv2.resize(img_rgb, target_size)
    input_tensor = tf.convert_to_tensor(np.expand_dims(img_resized, 0), dtype=tf.float32)

    # ================================
    # 5. Inferencia
    # ================================
    if infer is not None:
        print("\nEjecutando inferencia...")
        try:
            input_name = list(infer.structured_input_signature[1].keys())[0]
            outputs = infer(**{input_name: input_tensor})
            print("Claves de salida:", list(outputs.keys()))

            # ================================
            # 6. Interpretación para clasificación (3 clases)
            # ================================
            # Toma el primer tensor de salida disponible y aplica softmax
            first_key = list(outputs.keys())[0]
            logits = outputs[first_key].numpy()
            if logits.ndim == 2 and logits.shape[0] == 1:
                probs = tf.nn.softmax(logits[0]).numpy()
            else:
                probs = tf.nn.softmax(logits).numpy().squeeze()

            # Nombres de clases desde las subcarpetas de src/
            class_names = None
            if os.path.isdir(DATA_DIR):
                class_names = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])

            if class_names and len(class_names) == probs.shape[-1]:
                for name, p in zip(class_names, probs):
                    print(f"{name}: {p*100:.2f}%")
                pred_idx = int(np.argmax(probs))
                print("Predicción:", class_names[pred_idx])
            else:
                print("Probabilidades:", probs)

        except Exception as e:
            print("Error durante la inferencia:", e)


## Entrenamiento con 3 clases (hamburguesa, omelette, pizza)
Este bloque usa las imágenes en `src/` y entrena un modelo con MobileNetV2.


In [8]:

# Configurar dataset desde carpeta local src/
import os, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

DATA_DIR = os.path.join(os.getcwd(), "src")
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
VAL_SPLIT = 0.2
SEED = 1337

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
)

class_names = train_ds.class_names
print("Clases:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 141: invalid continuation byte

In [ ]:

# Transfer learning: MobileNetV2 congelada + cabeza densa para 3 clases
base = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,), include_top=False, weights="imagenet"
)
base.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)
model_ft = keras.Model(inputs, outputs)

model_ft.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

EPOCHS = 5
history = model_ft.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)


In [ ]:

# Guardar el modelo en formato SavedModel para reutilizarlo en la celda del profesor
save_dir = os.path.join(os.getcwd(), "model_from_src")
try:
    tf.saved_model.save(model_ft, save_dir)
    print("SavedModel exportado en:", save_dir)
except Exception as e:
    print("Error al guardar SavedModel:", e)


## Inferencia con clases nuevas
Usa la celda del profesor para cargar el SavedModel, y esta celda para mapear probabilidades a nombres de clase.


In [ ]:

# Si ya tienes "infer" del SavedModel cargado, interpreta la salida como softmax y mapea a class_names.
import numpy as np
import cv2

def predict_with_infer(img_path, target_size=(224,224)):
    if "infer" not in globals() or infer is None:
        print("Primero carga el modelo (firma 'serving_default').")
        return
    img = cv2.imread(img_path)
    if img is None:
        print("No se pudo leer la imagen:", img_path)
        return
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, target_size)
    input_tensor = tf.convert_to_tensor(np.expand_dims(img_resized, 0), dtype=tf.float32)

    input_name = list(infer.structured_input_signature[1].keys())[0]
    outputs = infer(**{input_name: input_tensor})

    # Intenta tomar el primer tensor de salida disponible
    first_key = list(outputs.keys())[0]
    logits = outputs[first_key].numpy()
    if logits.ndim == 2 and logits.shape[0] == 1:
        probs = tf.nn.softmax(logits[0]).numpy()
    else:
        probs = tf.nn.softmax(logits).numpy().squeeze()

    if 'class_names' in globals():
        for i,p in enumerate(probs):
            print(f"{class_names[i]}: {p*100:.2f}%")
        pred_idx = int(np.argmax(probs))
        print("Predicción:", class_names[pred_idx])
    else:
        print("Probabilidades:", probs)

# Ejemplo de uso (ajusta a una imagen de tu carpeta):
# predict_with_infer(os.path.join(DATA_DIR, 'hamburguesa', 'cheeseburger.jpg'))


## Cargar modelo local (alternativa a la celda del profesor)
Carga el SavedModel exportado en `model_from_src` y expone la firma `infer`.
